<a href="https://colab.research.google.com/github/quantumseminar/textbook/blob/main/05deeplearning/01neuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 ニューロン

## 1.1 ニューラルネットワーク
ディープラーニングでは主にニューラルネットワークと呼ばれるものを利用します。ここでは、PyTorch（パイトーチ）というツールを使って順番にニューラルネットワークを利用しての計算を学びます。


PyTorchのインストール
インストールはシンプルです。pypiでインストールします。

In [1]:
pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 949.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 3.8 MB/s eta 0:00:00


ニューロンの単体ユニット
まずは複雑なニューラルネットワークに行く前に、単純な形状を学びます。ニューラルネットワークは脳の神経を単純化したニューロンと呼ばれるユニットがベースになっています。

単一のニューロンの線形ユニットを見てみます。これは、

$$
𝑦=𝑤𝑥+𝑏
$$

とかけます。wを重みと言います。xは入力値。bはバイアスと呼ばれます。yは出力値です。単一のニューロンのユニットでは、wとxをかけてbを足すことで出力値が得られます。

![../img/neuron01.jpg](../img/neuron01.jpg)

まずは通常のPythonコードでyの計算をしてみます。

In [2]:
x = 2
w = 3
b = 4

y = w*x + b
print(y)

10


x=2からy=10という値が出力されました。xの入力値を変更するとyが変化します。

PyTorchをつかって計算をする際にはちょっと複雑です。

In [3]:
#ツールを
import torch
import torch.nn as nn

#入力値xの個数を指定。今回はxひとつだけ使うので、1にします。
num_input = 1

#ニューロンの数を指定
num_neuron = 1

#単体ユニットを作る
fc = nn.Linear(num_input, num_neuron)

#wとbを設定します。ちょっと設定方法は難しいかもしれませんが今回は覚えてしまってください。
fc.weight = nn.Parameter(torch.tensor([[3.]]))
fc.bias = nn.Parameter(torch.tensor([4.]))

#設定したwとbを確認します。
print(fc.weight)
print(fc.bias)
b = 4

y = w*x + b
print(y)

Parameter containing:
tensor([[3.]], requires_grad=True)
Parameter containing:
tensor([4.], requires_grad=True)
10


少し複雑でしたが、x=2を入れてみて計算をしてみましょう。xの設定もちょっと特殊ですが、今回は覚えてしまってください。

In [4]:
#xを決める
x = torch.tensor([[2]], dtype=torch.float32)

#yを計算する
y = fc(x)
print(y)

tensor([[10.]], grad_fn=<AddmmBackward0>)


ということで、上の一部に10が見れました。これを少しずつ規模を大きくしていきます。

## 1.2 ニューロンの単一ユニットで複数入力
先ほどは入力のxはひとつだけを使いました。通常は複数の入力値を使います。例えば、入力の値を３つ用意し、𝑥0と𝑥1と𝑥2を準備します。かける𝑤も𝑤0と𝑤1と𝑤2の３つを用意します。bは入力が増えても一つのままです。

$$
𝑦=𝑤0∗𝑥0+𝑤1∗𝑥1+𝑤2∗𝑥2+𝑏
$$

![../img/neuron02.jpg](../img/neuron02.jpg)

まずは通常のPythonコードでyの計算をしてみます。

In [5]:
x0 = 2
x1 = 3
x2 = 4

w0 = 5
w1 = 6
w2 = 7

b = 8

y = w0*x0 + w1*x1 + w2*x2 + b
print(y)

64


64という値が出力されました。 続いて同じ計算をPyTorchをつかって計算してみます。

In [6]:
#ツールを読み込み
import torch
import torch.nn as nn

#入力値xの個数を3に設定
num_input = 3

#ニューロンの数を指定
num_neuron = 1

#単体ユニットを作る
fc = nn.Linear(num_input, num_neuron)

#wとbを設定します。今回wは三つ設定します。
fc.weight = nn.Parameter(torch.tensor([[5.,6.,7.]]))
fc.bias = nn.Parameter(torch.tensor([8.]))

#xを決める。今回は2,3,4の三つを入れます。
x = torch.tensor([[2,3,4]], dtype=torch.float32)

#yを計算する
y = fc(x)
print(y)

tensor([[64.]], grad_fn=<AddmmBackward0>)


同じように答え64が得られました。次はもっと規模を大きくしてみます。

## 1.3 単一レイヤー
これまで入力は複数で、ニューロンはひとつでした。入力の数を変更せずに、ニューロンの数を二つに増やしてみます。そうすると重みwが増えます。

$$
𝑦0=𝑤00∗𝑥0+𝑤10∗𝑥1+𝑤20∗𝑥2+𝑏
$$

$$
𝑦1=𝑤01∗𝑥0+𝑤11∗𝑥1+𝑤21∗𝑥2+𝑏
$$

![../img/neuron03.jpg](../img/neuron03.jpg)

まずは通常のPythonコードでyの計算をしてみます。

In [7]:
x0 = 1
x1 = 2
x2 = 3

w00 = 4
w10 = 5
w20 = 6

w01 = 7
w11 = 8
w21 = 9

b = 10

y0 = w00*x0 + w10*x1 + w20*x2 + b
print("y0=", y0)

y1 = w01*x0 + w11*x1 + w21*x2 + b
print("y1=", y1)

y0= 42
y1= 60


y0は42。y1は60の値となりました。同様にPyTorchで行ってみます。

In [8]:
#ツールを読み込み
import torch
import torch.nn as nn

#入力値xの個数を3に設定
num_input = 3

#ニューロンの数を2に変更
num_neuron = 2

#単体ユニットを作る
fc = nn.Linear(num_input, num_neuron)

#wとbを設定します。今回wは三つ設定します。
fc.weight = nn.Parameter(torch.tensor([[4.,5.,6.],[7.,8.,9.]]))
fc.bias = nn.Parameter(torch.tensor([10.]))

#xを決める。今回は1,2,3の三つを入れます。
x = torch.tensor([[1,2,3]], dtype=torch.float32)

#yを計算する
y = fc(x)
print(y)

tensor([[42., 60.]], grad_fn=<AddmmBackward0>)


先ほどと同じように、42と60が得られました。このように複数のニューロンを同時に計算する場合、一つのレイヤーとして扱います。